In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from numpy import mean
from matplotlib.pyplot import figure

def process_data(data):
    data = numerize_sex(data)
    data = numerize_embarked(data)
    data = create_rich_category(data)
    data = update_rich_category(data)
    data = create_poor_category(data)
    data = create_middleclass_category(data)
    data = create_rich_age_selection(data)
    data = create_middleclass_age_selection(data)
    data = create_poor_age_selection(data)
    data = create_familysize_category(data)
    data = create_familycombo_category(data)
    data = drop_passengerid_and_class(data)
    data = create_title_category(data)
    data = create_probability_category(data)
    data = create_titleembarked_category(data)
    data = create_ticketcategory_category(data)
    data = drop_remaining_irrelevant_columns(data)
    return data

def numerize_sex(data):
    data['Sex'] = data['Sex'].map({'male': 0, 'female':1})
    return data

def numerize_embarked(data):
    data['Embarked'] = data['Embarked'].map({'S':0, 'C':1, 'Q':2, '':3})
    return data

def create_rich_category(data):
    data['Rich'] = data['Pclass']
    data.loc[data['Fare']>100, 'Rich'] = 1
    data.loc[data['Fare']<100, 'Rich'] = 0
    return data

def update_rich_category(data):
    data.loc[data['Fare']>76, 'Rich'] = 1
    return data

def create_poor_category(data):
    data['Poor'] = data['Rich']
    data.loc[data['Fare']<10, 'Poor'] = 1
    data.loc[data['Fare']>=10, 'Poor'] = 0
    return data

def create_middleclass_category(data):
    data['MiddleClass'] = df['Rich']
    data.loc[data['Fare']>10, 'MiddleClass'] = 1
    data.loc[data['Rich']==1, 'MiddleClass'] = 0
    data.loc[data['Poor']==1, 'MiddleClass'] = 0
    return data

def create_rich_age_selection(data):
    data['AgeSelectionWhenRich'] = data['Rich']
    richNs = (data['Rich']== 1) & (data['Age']<28)
    nonRichNs = (data['Rich']== 1) & (data['Age']>28)
    data.loc[richNs, 'AgeSelectionWhenRich'] = 1
    data.loc[nonRichNs, 'AgeSelectionWhenRich'] = 0
    return data

def create_middleclass_age_selection(data):
    data['AgeSelectionWhenMiddle'] = data['MiddleClass']
    middleClassNs = (data['MiddleClass']== 1) & (data['Age']<6)
    nonMiddleClassNs = (data['MiddleClass']== 1) & (data['Age']>6)
    data.loc[middleClassNs, 'AgeSelectionWhenMiddle'] = 1
    data.loc[nonMiddleClassNs, 'AgeSelectionWhenMiddle'] = 0
    return data

def create_poor_age_selection(data):
    data['AgeSelectionWhenPoor'] = data['Poor']
    poorNs = (data['Poor']== 1) & (data['Age']<14)
    nonPoorNs = (data['Poor']== 1) & (data['Age']>14)
    data.loc[poorNs, 'AgeSelectionWhenPoor'] = 1
    data.loc[nonPoorNs, 'AgeSelectionWhenPoor'] = 0
    return data

def create_familysize_category(data):
    data['FamilySize'] = data['SibSp'] + data['Parch']
    return data

def create_familycombo_category(data):
    data['FamilyCombo'] = data['Poor']
    richFamilyCombo = (data['FamilySize']== 4) & (data['Rich']==1)
    poorFamilyCombo = (data['FamilySize']== 6) & (data['Poor']==1)
    data.loc[data['Poor']==1, 'FamilyCombo'] = 0
    data.loc[data['Poor']==0, 'FamilyCombo'] = 0
    data.loc[richFamilyCombo, 'FamilyCombo'] = 1
    data.loc[poorFamilyCombo, 'FamilyCombo'] = 1
    return data
    
def drop_passengerid_and_class(data):
    data.drop('PassengerId', axis=1, errors='ignore', inplace=True)
#     data.drop('Pclass', axis=1, errors='ignore', inplace=True)
    return data

def create_title_category(data):
    data['Title'] = data['Name']
    data.loc[data['Name'].str.find('Mr.')>-1, 'Title'] = 0
    data.loc[data['Name'].str.find('Master.')>-1, 'Title'] = 1
    data.loc[data['Name'].str.find('Miss.')>-1, 'Title'] = 2
    data.loc[data['Name'].str.find('Mrs.')>-1, 'Title'] = 3
    data.loc[data['Name'].str.find('Dr.')>-1, 'Title'] = 4
    data.loc[data['Name'].str.find('Rev.')>-1, 'Title'] = 5
    data.loc[data['Name'].str.find('Don.')>-1, 'Title'] = 6
    data.loc[data['Name'].str.find('Mme.')>-1, 'Title'] = 7
    data.loc[data['Name'].str.find('Major.')>-1, 'Title'] = 8
    data.loc[data['Name'].str.find('Ms.')>-1, 'Title'] = 9
    data.loc[data['Name'].str.find('Lady.')>-1, 'Title'] = 10
    data.loc[data['Name'].str.find('Col.')>-1, 'Title'] = 11
    data.loc[data['Name'].str.find('Sir.')>-1, 'Title'] = 12
    data.loc[data['Name'].str.find('Mlle.')>-1, 'Title'] = 13
    data.loc[data['Name'].str.find('Countess.')>-1, 'Title'] = 14
    data.loc[data['Name'].str.find('Capt.')>-1, 'Title'] = 15
    data.loc[data['Name'].str.find('Jonkheer.')>-1, 'Title'] = 16
    data.loc[data['Name'].str.find('Dona.')>-1, 'Title'] = 2
    return data

def create_probability_category(data):
    data['Probability'] = data['Title']
    data.loc[data['Title'].astype(int)>=0, 'Probability'] = 0

    richPassengersFromSouthampton = (data['Rich']==1) & (data['Embarked']==0) & ((data['Title']==1) | (data['Title']==4) | (data['Title']==14))
    richPassengersFromCherbourg = (data['Rich']==1) & (data['Embarked']==1) & ((data['Title']==2) | (data['Title']==3))
    middleClassPassengersFromSouthampton = (data['MiddleClass']==1) & (data['Embarked']==0) & ((data['Title']==9))
    middleClassPassengersFromCherbourg = (data['MiddleClass']==1) & (data['Embarked']==1) & ((data['Title']==1) | (data['Title']==7) | (data['Title']==10) | (data['Title']==11) | (data['Title']==12) | (data['Title']==13))
    poorPassengersFromCherbourg = (data['Poor']==1) & (data['Embarked']==1) & ((data['Title']==1) | (data['Title']==2) | (data['Title']==3))

    data.loc[richPassengersFromSouthampton, 'Probability'] = 1
    data.loc[richPassengersFromCherbourg, 'Probability'] = 1
    data.loc[middleClassPassengersFromSouthampton, 'Probability'] = 1
    data.loc[middleClassPassengersFromCherbourg, 'Probability'] = 1
    data.loc[poorPassengersFromCherbourg, 'Probability'] = 1

    richMrSurvivalFromSouthampton = (data['Rich']==1) & (data['Embarked']==0) & (data['Title']==0) & (data['Sex']==0) & ((data['Cabin']=='C93') | (data['Cabin']=='B96') | (data['Cabin']=='B98') | ((data['Age']>=36) & (data['Age']<=38) & (data['Fare']>=85) & (data['Fare']<=125)))
    richMissSurvivalFromSouthampton = (data['Rich']==1) & (data['Embarked']==0) & (data['Title']==2) & (data['Sex']==1) & ((data['Cabin']=='C23') | (data['Cabin']=='C25') | (data['Cabin']=='C27') | (data['Cabin']=='B77') | (data['Cabin']=='C99') | (data['Cabin']=='D7') | (data['Cabin']=='C7') | (data['Cabin']=='B96') | (data['Cabin']=='B98') | (data['Cabin']=='B79') | (data['Cabin']=='B73') | (data['Cabin']=='E68') | (data['Cabin']=='C125') | (data['Cabin']=='D9') | (data['Cabin']=='B5') | (data['FamilySize']!=3))
    richMrsSurvivalFromSouthampton = (data['Rich']==1) & (data['Embarked']==0) & (data['Title']==3) & (data['Sex']==1) & ((data['Cabin']=='C83') | (data['Cabin']=='C125') | (data['Cabin']=='C93') | (data['Cabin']=='E67') | (data['Cabin']=='B96') | (data['Cabin']=='B98') | (data['Cabin']=='D11') | (data['Cabin']=='B3') | (data['Cabin']=='B69') | (data['FamilySize']<3))
    richMrSurvivalFromCherbourg = (data['Rich']==1) & (data['Embarked']==1) & (data['Title']==0) & (data['Sex']==0) & ((data['Cabin']=='C92') | (data['Cabin']=='B49') | (data['Cabin']=='C70') | (data['Cabin']=='B41') | (data['Cabin']=='D33') | (data['Cabin']=='B51') | (data['Cabin']=='B53') | (data['Cabin']=='B55') | (data['Cabin']=='D49') | (data['Cabin']=='B101') | (data['Fare']>=250))

    middleClassMrSurvivalFromSouthampton = (data['MiddleClass']==1) & (data['Embarked']==0) & (data['Title']==0) & (data['Sex']==0) & (data['Parch']<=1) & (data['FamilySize']<=2) & ((data['Cabin']=='C92') | (data['Cabin']=='D56') | (data['Cabin']=='A6') | (data['Cabin']=='C52') | (data['Cabin']=='D35') | (data['Cabin']=='C106') | (data['Cabin']=='E12') | (data['Cabin']=='E25') | (data['Cabin']=='D19') | (data['Cabin']=='A23') | (data['Cabin']=='B20') | (data['Cabin']=='E24') | (data['Cabin']=='C126') | (data['Cabin']=='E8') | (data['Cabin']=='D45') | (data['Cabin']=='E17'))
    middleClassMasterSurvivalFromSouthampton = (data['MiddleClass']==1) & (data['Embarked']==0) & (data['Title']==1) & (data['Sex']==0) & ((data['FamilySize']<=3) | (data['FamilySize']==6))
    middleClassMissSurvivalFromSouthampton = (data['MiddleClass']==1) & (data['Embarked']==0) & (data['Title']==2) & (data['Sex']==1) & (((data['Cabin']=='G6') | (data['Cabin']=='C103') | (data['Cabin']=='E101') | (data['Cabin']=='D47') | (data['Cabin']=='F33') | (data['Cabin']=='E33') | (data['Cabin']=='B22') | (data['Cabin']=='F4') | (data['Cabin']=='D28') | (data['Cabin']=='B42')) | (data['FamilySize']<=3) | (data['FamilySize']==6))
    middleClassMrsSurvivalFromSouthampton = (data['MiddleClass']==1) & (data['Embarked']==0) & (data['Title']==3) & (data['Sex']==1) & (((data['Cabin']=='C123') | (data['Cabin']=='F33') | (data['Cabin']=='C2') | (data['Cabin']=='E33') | (data['Cabin']=='D') | (data['Cabin']=='G6') | (data['Cabin']=='D21') | (data['Cabin']=='C101') | (data['Cabin']=='E44') | (data['Cabin']=='C126') | (data['Cabin']=='B20') | (data['Cabin']=='E8') | (data['Cabin']=='E121') | (data['Cabin']=='D17') | (data['Cabin']=='D35')))
    middleClassDrSurvivalFromSouthampton = (data['MiddleClass']==1) & (data['Embarked']==0) & (data['Title']==4) & (data['Age']>=23) & (data['Fare']>=25)
    middleClassMajorSurvivalFromSouthampton = (data['MiddleClass']==1) & (data['Embarked']==0) & (data['Title']==8) & (data['Fare']>=30)

    middleClassMrSurvivalFromCherbourg = (data['MiddleClass']==1) & (data['Embarked']==1) & (data['Title']==0) & (data['Sex']==0) & ((data['Cabin']=='D10') | (data['Cabin']=='D12') | (data['Cabin']=='A31') | (data['Cabin']=='E50') | (data['Cabin']=='C47') | (data['Cabin']=='C148') )
    middleClassMissSurvivalFromCherbourg = (data['MiddleClass']==1) & (data['Embarked']==1) & (data['Title']==2) & (data['Sex']==1) & (((data['Cabin']=='F E69') | (data['Cabin']=='E36') | (data['Cabin']=='B18') | (data['Cabin']=='B39')))
    middleClassMrsSurvivalFromCherbourg = (data['MiddleClass']==1) & (data['Embarked']==1) & (data['Title']==3) & (data['Sex']==1) & (((data['Cabin']=='C85') | (data['Cabin']=='B4') | (data['Cabin']=='D37') | (data['Cabin']=='D') | (data['Cabin']=='B18')))
    middleClassDrSurvivalFromCherbourg = (data['MiddleClass']==1) & (data['Embarked']==1) & (data['Title']==4) & (data['Age']>=23) & (data['Fare']>=25)

    poorMrSurvivalFromSouthampton = (data['Poor']==1) & (data['Embarked']==0) & (data['Title']==0) & (data['Sex']==0) & (data['Parch']==0) & (data['FamilySize']<=1) & (data['SibSp']<=1)
    poorMrsSurvivalFromSouthampton = (data['Poor']==1) & (data['Embarked']==0) & (data['Title']==3) & (data['Sex']==1) & (data['SibSp']==0) & (data['FamilySize']>=0) & (data['Fare']>=8.5)
    poorMrSurvivalFromCherbourg = (data['Poor']==1) & (data['Embarked']==1) & (data['Title']==0) & (data['Sex']==0) & (data['FamilySize']==0) & (data['Age']>=20) & (data['Age']<=29) & (data['Fare']>=7.2) & (data['Fare']<=7.9)

    data.loc[richMrSurvivalFromSouthampton, 'Probability'] = 1
    data.loc[richMissSurvivalFromSouthampton, 'Probability'] = 1
    data.loc[richMrsSurvivalFromSouthampton, 'Probability'] = 1
    data.loc[richMrSurvivalFromCherbourg, 'Probability'] = 1
    data.loc[middleClassMrSurvivalFromSouthampton, 'Probability'] = 1
    data.loc[middleClassMasterSurvivalFromSouthampton, 'Probability'] = 1
    data.loc[middleClassMissSurvivalFromSouthampton, 'Probability'] = 1
    data.loc[middleClassMrsSurvivalFromSouthampton, 'Probability'] = 1
    data.loc[middleClassDrSurvivalFromSouthampton, 'Probability'] = 1
    data.loc[middleClassMajorSurvivalFromSouthampton, 'Probability'] = 1
    data.loc[middleClassMrSurvivalFromCherbourg, 'Probability'] = 1
    data.loc[middleClassMissSurvivalFromCherbourg, 'Probability'] = 1
    data.loc[middleClassMrsSurvivalFromCherbourg, 'Probability'] = 1
    data.loc[middleClassDrSurvivalFromCherbourg, 'Probability'] = 1
    data.loc[poorMrSurvivalFromSouthampton, 'Probability'] = 1
    data.loc[poorMrsSurvivalFromSouthampton, 'Probability'] = 1
    data.loc[poorMrSurvivalFromCherbourg, 'Probability'] = 1
    return data

def create_titleembarked_category(data):
    data['TitleEmbarkedSelection'] = data['Poor']

    titleSouthamptonRich = (data['Rich']== 1) & (data['Embarked']==0) & ((data['Title']==1) | (data['Title']==2) | (data['Title']==3) | (data['Title']==4) | (data['Title']==14))
    titleCherbourgRich = (data['Rich']== 1) & (data['Embarked']==1) & ((data['Title']==2) | (data['Title']==3))
    titleSouthamptonMiddleClass = (data['MiddleClass']== 1) & (data['Embarked']==0) & (data['Title']==9)
    titleCherbourgMiddleClass = (data['MiddleClass']== 1) & (data['Embarked']==1) & ((data['Title']==1) | (data['Title']==7) | (data['Title']==10) | (data['Title']==11) | (data['Title']==12) | (data['Title']==13))
    titleCherbourgPoor = (data['Poor']== 1) & (data['Embarked']==1) & ((data['Title']==1) | (data['Title']==2) | (data['Title']==3))

    data.loc[data['Poor']==1, 'TitleEmbarkedSelection'] = 0
    data.loc[data['Poor']==0, 'TitleEmbarkedSelection'] = 0

    data.loc[titleSouthamptonRich, 'TitleEmbarkedSelection'] = 1
    data.loc[titleCherbourgRich, 'TitleEmbarkedSelection'] = 1
    data.loc[titleSouthamptonMiddleClass, 'TitleEmbarkedSelection'] = 1
    data.loc[titleCherbourgMiddleClass, 'TitleEmbarkedSelection'] = 1
    data.loc[titleCherbourgPoor, 'TitleEmbarkedSelection'] = 1
    return data

def create_ticketcategory_category(data):
    data['TicketCategory'] = data['Ticket']

    data.loc[data['Ticket'].str.find('STON')>-1, 'TicketCategory'] = 5
    data.loc[data['Ticket'].str.find('SOTON')>-1, 'TicketCategory'] = 6
    data.loc[data['Ticket'].str.find('SC.')>-1, 'TicketCategory'] = 7
    data.loc[data['Ticket'].str.find('LINE')>-1, 'TicketCategory'] = 8
    data.loc[data['Ticket'].str.find('C.A')>-1, 'TicketCategory'] = 9
    data.loc[data['Ticket'].str.find('Paris')>-1, 'TicketCategory'] = 10
    data.loc[data['Ticket'].str.find('PARIS')>-1, 'TicketCategory'] = 11
    data.loc[data['Ticket'].str.find('PC')>-1, 'TicketCategory'] = 12
    data.loc[data['Ticket'].str.find('PP')>-1, 'TicketCategory'] = 13
    data.loc[data['Ticket'].str.find('W./C')>-1, 'TicketCategory'] = 14
    data.loc[data['Ticket'].str.find('CA')>-1, 'TicketCategory'] = 15
    data.loc[data['Ticket'].str.find('S.O./P.P.')>-1, 'TicketCategory'] = 16
    data.loc[data['Ticket'].str.find('A/5')>-1, 'TicketCategory'] = 17
    data.loc[data['Ticket'].str.find('A/4')>-1, 'TicketCategory'] = 18
    data.loc[data['Ticket'].str.find('C ')>-1, 'TicketCategory'] = 19
    data.loc[data['Ticket'].str.find('SO')>-1, 'TicketCategory'] = 20
    data.loc[data['Ticket'].str.find('SC')>-1, 'TicketCategory'] = 21
    data.loc[data['Ticket'].str.find('F.C.C')>-1, 'TicketCategory'] = 22
    data.loc[data['Ticket'].str.find('F.C.')>-1, 'TicketCategory'] = 23
    data.loc[data['Ticket'].str.find('A./5')>-1, 'TicketCategory'] = 24
    data.loc[data['Ticket'].str.find('A.5')>-1, 'TicketCategory'] = 25
    data.loc[data['Ticket'].str.find('Fa')>-1, 'TicketCategory'] = 26
    data.loc[data['Ticket'].str.find('WE/P')>-1, 'TicketCategory'] = 27
    data.loc[data['Ticket'].str.find('S.O.C')>-1, 'TicketCategory'] = 28
    data.loc[data['Ticket'].str.find('W.E.P')>-1, 'TicketCategory'] = 29
    data.loc[data['Ticket'].str.find('S.P')>-1, 'TicketCategory'] = 30
    data.loc[data['Ticket'].str.find('S.C')>-1, 'TicketCategory'] = 31
    data.loc[data['Ticket'].str.find('A/S')>-1, 'TicketCategory'] = 32
    data.loc[data['Ticket'].str.find('A4.')>-1, 'TicketCategory'] = 33
    data.loc[data['Ticket'].str.find('S.O.P.')>-1, 'TicketCategory'] = 34


    sevenNumbers = (data['TicketCategory'].str.isnumeric()) & (data['TicketCategory'].str.len()==7)
    sixNumbers = (data['TicketCategory'].str.isnumeric()) & (data['TicketCategory'].str.len()==6) 
    fiveNumbers = (data['TicketCategory'].str.isnumeric()) & (data['TicketCategory'].str.len()==5)
    fourNumbers = (data['TicketCategory'].str.isnumeric()) & (data['TicketCategory'].str.len()==4)
    threeNumbers = (data['TicketCategory'].str.isnumeric()) & (data['TicketCategory'].str.len()==3)

    data.loc[sevenNumbers, 'TicketCategory'] = 0
    data.loc[sixNumbers, 'TicketCategory'] = 1
    data.loc[fiveNumbers, 'TicketCategory'] = 2
    data.loc[fourNumbers, 'TicketCategory'] = 3 
    data.loc[threeNumbers, 'TicketCategory'] = 4
    return data

def drop_remaining_irrelevant_columns(data):
#     data.drop(['Age','Ticket','Name','Cabin'], axis=1, errors='ignore', inplace=True)
    return data
    
def save_predictions(data, file_name):
    df = pd.read_csv('./data/gender_submission.csv')
    df['Survived'] = data
    return df.to_csv(file_name, index=False)

def correlation_graph(data):
    corr = data.corr()
    mask = np.triu(np.ones_like(corr, dtype=np.bool)) # Generate a mask for the upper triangle
    cmap = sns.diverging_palette(290, 10, as_cmap=True) # Generate a custom diverging colormap
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.6, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .9}) # Draw the heatmap with the mask and correct aspect ratio
